## Triangular Arbitrage Case

#### Motivation: 

Extracting arbitrage opportunities from the market using the triangular arbitrage method.

Steps:
1. Extracting listed pairs from the binance
2. Extracting the order book for each pair
3. Calculating the arbitrage opportunity for each pair
4. Extracting the arbitrage opportunities from the market

Libaries used in this case:

1. requests:
    - Used for extracting pairs listed in the binance
2. websocket:
    - Used  for extracting the order book for each pair
    - Payload:
    ```python
             payload: {
             "u":400900217,     // order book updateId
             "s":"BNBUSDT",     // symbol
             "b":"25.35190000", // best bid price
             "B":"31.21000000", // best bid qty
             "a":"25.36520000", // best ask price
             "A":"40.66000000"  // best ask qty
            }
    ```
    - Best bid price: The highest price that a buyer is willing to pay for an asset
    - Best ask price: The lowest price that a seller is willing to sell for an asset

#### Functions created in this case

1. get_binance_currencies()
    - Extracts the pairs listed in the binance
    -function:
    ```python
    import requests

    def get_binance_currencies():
        # Binance API endpoint for exchange information
        endpoint = "https://api.binance.com/api/v3/exchangeInfo"

        # Make a GET request to the endpoint
        response = requests.get(endpoint)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            exchange_info = response.json()

            base_currency = [symbol_info['baseAsset'] for symbol_info in exchange_info['symbols']]
            quote_currency = [symbol_info['quoteAsset'] for symbol_info in exchange_info['symbols']]
            trading_pairs = [symbol_info['symbol'] for symbol_info in exchange_info['symbols']]

            return trading_pairs,base_currency,quote_currency
        else:
            print(f"Failed to retrieve currencies. Status code: {response.status_code}")
            return None
    ```
    - Expected output:

| pair    | base  | quote |
|---------|-------|-------|
| ETHBTC  | ETH   | BTC   |
| LTCBTC  | LTC   | BTC   |
| BNBBTC  | BNB   | BTC   |
| NEOBTC  | NEO   | BTC   |
| QTUMETH | QTUM  | ETH   |



2. get_triangle:
    - to make easier and spend less time to find triangluar arbitrage opportunites.
    - function:
    ```python
       def get_triangle(sample):
            triangles = []
            for currency1 in sample['base'].unique():
                for currency2 in sample.loc[sample['base']==currency1,'quote'].unique():
                    for currency3 in sample.loc[sample['base']==currency2,'quote'].unique():
                        if sample.loc[(sample['base']==currency1) & (sample['quote']==currency3)].shape[0] > 0:
                            triangles.append([currency1+currency2, currency2+currency3, currency1+currency3])
            return triangles
    ```
    - Expected output:
    ```python
    [['ETHBTC', 'BTCUSDT', 'ETHUSDT'],
     ['ETHBTC', 'BTCUSDT', 'ETHUSDT']]
    ```

3. check_triangular_arbitrage_opportunity
    - this is the main function to calculate the arbitrage opportunity for each pair
    - takas the list of triangles as an input
    - gives the highest arbitrage opportunity and the best triangle to trade as print and returns dataframe
    - function:
```python

"""
This module contains functions for checking the triangular arbitrage opportunities.
Basically, the functions in this module are used to check if there is a triangular arbitrage opportunity
and print the result and record the result in a dataframe.

The inputs of the functions are the dataframe created by data_processing.py.
The outputs of the functions are the updated dataframe.
"""

def check_triangular_arbitrage_opportunity(triangles):

    global df 
    highest_arbitrage = 0
    best_triangle = None
    
    # Iterate over each triangle
    for triangle in triangles:
        
        pair1 = triangle[0]
        pair2 = triangle[1]
        pair3 = triangle[2]

        # Pass null values if the pair is not in the book ticker
        if pair1  not in book_ticker_df['Symbol'].values:
            continue
        if pair2 not in book_ticker_df['Symbol'].values:
            continue
        if pair3 not in book_ticker_df['Symbol'].values:
            continue

        # Determine the rate for the first method
        ask_AB = book_ticker_df.loc[book_ticker_df['Symbol']==pair1,'Ask Price'].values[0]
        ask_BC = book_ticker_df.loc[book_ticker_df['Symbol']==pair2,'Ask Price'].values[0]
        ask_AC = book_ticker_df.loc[book_ticker_df['Symbol']==pair3,'Ask Price'].values[0]

        rate_1 = ask_AC/ask_AB * ask_BC

        # Determine the rate for the second method
        bid_AB = book_ticker_df.loc[book_ticker_df['Symbol']==pair1,'Bid Price'].values[0]
        bid_BC = book_ticker_df.loc[book_ticker_df['Symbol']==pair2,'Bid Price'].values[0]
        bid_AC = book_ticker_df.loc[book_ticker_df['Symbol']==pair3,'Bid Price'].values[0]

        rate_2 = bid_AB * bid_BC/bid_AC

        actual_rate = max(rate_1,rate_2)

        # Find the highest arbitrage opportunity
        """ actual_rate should be greater than 1.003 to make a profit
            actual_rate should be less than 2 to avoid misinterpretation of the decimal places
        """
        if (rate_1>1.003 or rate_2>1.003) and actual_rate <2:
            highest_arbitrage = max(actual_rate,highest_arbitrage)
            best_triangle = triangle
            action = "Buy" if rate_1>rate_2 else "Sell"

            if rate_1>rate_2:
                best_pair_1 = pair1
                best_pair_2 = pair2
                best_pair_3 = pair3
            else:
                best_pair_1 = pair1
                best_pair_2 = pair2
                best_pair_3 = pair3

    # Print the arbitrage opportunity
    if highest_arbitrage > 1:
        print("Arbitrage Opportunity Detected!")
        print(f"Highest Arbitrage: {float(highest_arbitrage)}")
        print(f"Triangle: {best_triangle}")
        print(f"Action: {action}")

        
        # Record the arbitrage opportunity to a dataframe
        new_row = pd.DataFrame([[highest_arbitrage, best_pair_1,
                                 best_pair_2,best_pair_3, action]], columns=df.columns)
        df = pd.concat([df, new_row], ignore_index=True)
        df.drop_duplicates(inplace=True)
```

4. Message processing function:
- Data is processed in the following way:
    - The data is processed in the *process_book_ticker* function
    - The function updates the DataFrame with the latest bid and ask prices
    - The function is called in the *on_message* function
- DataFrame is updated frequently: (DELTA FORMAT)
    - The DataFrame is updated in the *update_dataframe* function
    - The function checks if the symbol is already in the DataFrame
    - If the symbol is already in the DataFrame, the function updates the existing row
    - If the symbol is not in the DataFrame, the function inserts a new row

- Function:
```python

"""
This module contains functions for processing the data received from the Binance WebSocket.
Basically, the functions in this module are used to create dataframe to record the book ticker data and
update the dataframe when new data is received.

The inputs of the functions are the data received from the Binance WebSocket.
The outputs of the functions are the updated dataframe.
output columns: Symbol, Bid Price, Ask Price
Symbol: the symbol of the coin String
Bid Price: the bid price of the coin Float (Decimal:20 places)
Ask Price: the ask price of the coin Float (Decimal:20 places)

Modules used:
pandas: to create and update the dataframe
decimal: to set the precision of the float numbers
"""
def process_book_ticker(data):
    
    symbol = data["s"]
    bid_price = Decimal(data["b"])
    ask_price = Decimal(data["a"])

    # Update or insert the data in the DataFrame
    update_dataframe(symbol, bid_price, ask_price)

def update_dataframe(symbol, bid_price, ask_price):
   
    global book_ticker_df

    # Check if the symbol is already in the DataFrame
    if symbol in book_ticker_df["Symbol"].values:
        # Update existing row
        book_ticker_df.loc[book_ticker_df["Symbol"] == symbol, ["Bid Price", "Ask Price"]] = [bid_price, ask_price]
    else:
        # Insert a new row
        new_row = pd.DataFrame([[symbol, bid_price, ask_price]], columns=book_ticker_df.columns)
        book_ticker_df = pd.concat([book_ticker_df, new_row], ignore_index=True)
```

5. API is called in the following way:
- api is called using *on_message*, *on_error*, *on_close* functions
- function:
```python
import pandas as pd
import websocket
import json
from decimal import Decimal,getcontext

getcontext().prec = 20


BINANCE_WS_BASE_URL = "wss://stream.binance.com:9443/ws/"

# Initialize an empty DataFrame
columns = ["Symbol", "Bid Price", "Ask Price"]
book_ticker_df = pd.DataFrame(columns=columns)
columns_2 = ["Highest Arbitrage", "A","B","C", "Action"]
df = pd.DataFrame(columns=columns_2)

# Error print
def on_error(ws, error):

    print(error)

# Close print
def on_close(ws, close_status_code, close_msg):

    print("Connection Closed", close_status_code, close_msg)

def on_message(ws, message):
    
    # message comes from the WebSocket as a string, convert it to a dictionary
    # Then the data is processed and the arbitrage opportunity is calculated
    data = json.loads(message)
    process_book_ticker(data)
    check_triangular_arbitrage_opportunity(triangles)

    # Close the WebSocket connection after 10 messages
    """ Simple closing condition is defined, but it can be improved by adding a time condition
    """
    if len(df) > 10:
        ws.close()


def subscribe_to_book_tickers(pairs):

    # Construct the WebSocket URL for book tickers
    symbols = [f"{pair.lower()}@bookTicker" for pair in sample["pair"]]
    url = f"{BINANCE_WS_BASE_URL}{'/'.join(symbols)}"

    # Start the WebSocket connection
    ws = websocket.WebSocketApp(url, on_message=on_message, on_error=on_error, on_close=on_close)
    ws.run_forever()
    ws.close()


```
- the output is as follows:
```
Arbitrage Opportunity Detected!
Highest Arbitrage: 1.003
Triangle: ['ETHBTC', 'BTCUSDT', 'ETHUSDT']
Action: Buy
```